In [1]:
# This script converts the trained "model.h5" into OpenVINO MO.
# https://docs.openvino.ai/2022.3/openvino_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_TensorFlow.html#keras-h5 
import time
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from IPython.display import Markdown
from openvino.runtime import Core

2024-07-02 16:06:55.755239: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-02 16:06:55.757412: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-02 16:06:55.797393: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-02 16:06:55.798477: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 16:06:56.386916: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
model_h5_dir = Path("./files/A_20240630")
model_h5_path = model_h5_dir / "model.h5"

In [3]:
from metrics import iou, dice_coef, dice_loss, bce_dice_loss
from tensorflow.keras.utils import CustomObjectScope

# Load the model
with CustomObjectScope({
        'iou':iou,
        'dice_coef':dice_coef,
        'dice_loss':dice_loss,
        'bce_dice_loss': bce_dice_loss
    }):
    try:
        h5_model = tf.keras.models.load_model(model_h5_path)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Failed to load model: {e}")
        

Model loaded successfully.


In [4]:
saved_model_path =  model_h5_dir /'model'
tf.saved_model.save(h5_model,saved_model_path)


2024-07-02 16:07:07.441945: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2024-07-02 16:07:07.581047: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2024-07-02 16:07:07.719960: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2024-07-02

INFO:tensorflow:Assets written to: files/A_20240630/model/assets


INFO:tensorflow:Assets written to: files/A_20240630/model/assets


In [11]:
import openvino as ov
ov_model = ov.convert_model(saved_model_path, input=[[1, 256, 256, 3]]) 

In [12]:
prep = ov.preprocess.PrePostProcessor(ov_model)
input_name = ov_model.input().get_any_name()
print("input_name is", input_name)
prep.input(input_name).model().set_layout(ov.Layout("NCHW"))
prep.input(input_name).tensor().set_layout(ov.Layout("NCHW"))
ov_model = prep.build()


input_name is input_image


In [14]:
ov.save_model(ov_model, saved_model_path /'nanonet-fp32-shape-1-3-256-256-model.xml')